In [1]:
import numpy as np
import pandas as pd
import os
from glob import glob

import config
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials

sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(config.client_id, config.client_secret))

# Getting Spotify Data

In [12]:
playlists = glob("./Playlists/*")

In [20]:
master = {}
for p in playlists:
    master[p] = glob(p+"/Tracks/*")

In [27]:
playlist_dfs = []

for k, v in master.items():
    playlist = k
    tracks = []
    for track in v:
        tracks.append(pd.read_csv(track))
    pdf = pd.concat(tracks)
    pdf['Playlist'] = k
    
    playlist_dfs.append(pdf)
    

In [116]:
full = pd.concat(playlist_dfs)
full = full[full.Removed != 'Yes']
full = full.drop(['Playlist Index', 'Removed'], axis=1)


In [142]:
full.Title = full['Title'].str.replace("&#39;", "'")
full.Title = full['Title'].str.replace("&amp;", "&")
full.Title = full['Title'].str.replace('\([^)]*\)', "", regex=True)

full.Artist = full['Artist'].str.replace("&#39;", "'")
full.Artist = full['Artist'].str.replace("&amp;", "&")
full.Artist = full['Artist'].str.replace('\([^)]*\)', "", regex=True)

In [143]:
full.Playlist = full.Playlist.str.replace("./Playlists/", '')

In [144]:
full.head(50)

,Title,Album,Artist,Duration (ms),Rating,Play Count,Playlist,spotify_id
0,Just a Friend,Just A Friend - EP,Biz Markie,238000,0,6,Old ✌️,spotify:track:3yNVfRQlUPViUh8O2V9SQn
0,Love,Aijuswanaseing,Musiq Soulchild,304000,0,91,Old ✌️,spotify:track:6uSa5iCMwPr10Ftpz8w09b
0,Love Like This,Pocketful Of Sunshine,Natasha Bedingfield,222000,0,1,Old ✌️,spotify:track:49ppqF73rOz5Nyt1paRxIF
0,You Make Me Wanna...,My Way,Usher,219000,0,27,Old ✌️,spotify:track:47TqCCnEliDp8NRDyIQoQq
0,Miss Independent,Year Of The Gentleman,Ne-Yo,232000,0,35,Old ✌️,spotify:track:34ceTg8ChN5HjrqiIYCn9Q
0,Burn,Confessions (Special Edition),Usher,231000,0,2,Old ✌️,spotify:track:7z3N2W7Xz1t2G2sAO8wFVH
0,Over And Over,Suit,Nelly,253000,0,6,Old ✌️,spotify:track:2b8QQqf8Vwb7bfz22ja8Fk
0,We Belong Together,The Emancipation of Mimi,Mariah Carey,201000,0,53,Old ✌️,spotify:track:3LmvfNUQtglbTrydsdIqFU
0,So Into You,Tamia,Tamia,261000,0,38,Old ✌️,spotify:track:3c6afiysmB7OnxQzzSqRfD
0,If I Ain't Got You,The Diary Of Alicia Keys,Alicia Keys,228000,0,129,Old ✌️,spotify:track:3XVBdLihbNbxUwZosxcGuJ


In [145]:
def get_spotify_uri(row):
    artist = row['Artist']
    track = row['Title']
    
    query = 'artist: {} track: {}'.format(artist, track)
    items = sp.search(q=query, limit=1)['tracks']['items']
    
    if items != []:
        return items[0]['uri']
    else:
        return None

In [146]:
full['spotify_id'] = full.apply(lambda r: get_spotify_uri(r), axis=1)

In [131]:
full

,Title,Album,Artist,Duration (ms),Rating,Play Count,Playlist,spotify_id
0,Just a Friend,Just A Friend - EP,Biz Markie,238000,0,6,Old ✌️,spotify:track:3yNVfRQlUPViUh8O2V9SQn
0,Love,Aijuswanaseing,Musiq Soulchild,304000,0,91,Old ✌️,spotify:track:6uSa5iCMwPr10Ftpz8w09b
0,Love Like This,Pocketful Of Sunshine,Natasha Bedingfield,222000,0,1,Old ✌️,spotify:track:49ppqF73rOz5Nyt1paRxIF
0,You Make Me Wanna...,My Way,Usher,219000,0,27,Old ✌️,spotify:track:47TqCCnEliDp8NRDyIQoQq
0,Miss Independent,Year Of The Gentleman,Ne-Yo,232000,0,35,Old ✌️,spotify:track:34ceTg8ChN5HjrqiIYCn9Q
...,...,...,...,...,...,...,...,...
0,You're On,Adventure (Deluxe),Madeon,192000,0,1,❄🏠,None
0,Man X Woman,Majestic Casual - Chapter 2,Full Crate x Mar,279000,0,49,❄🏠,spotify:track:1adYhAnblyNTZtrPAuu5ZH
0,You & Me,Settle (The Remixes),Disclosure,282000,0,8,❄🏠,spotify:track:1snNAXmmPXCn0dkF9DaPWw
0,Hero,No Moon At All,Wildcat! Wildcat!,205000,0,18,❄🏠,spotify:track:3z4KpgwbY1c35mwIQbtpB0


In [147]:
full.spotify_id.isna().sum()

268

In [148]:
# export the missing ones to a csv
full[full['spotify_id'].isna()].to_csv('missing.csv', index=False)

In [151]:
full = full.dropna(subset=['spotify_id'])

In [179]:
full.to_csv('songs.csv', index=False)

# Creating Playlists

In [7]:
full = pd.read_csv('songs.csv')
full

,Title,Album,Artist,Duration (ms),Rating,Play Count,Playlist,spotify_id
0,Just a Friend,Just A Friend - EP,Biz Markie,238000,0,6,Old ✌️,spotify:track:3yNVfRQlUPViUh8O2V9SQn
1,Love,Aijuswanaseing,Musiq Soulchild,304000,0,91,Old ✌️,spotify:track:6uSa5iCMwPr10Ftpz8w09b
2,Love Like This,Pocketful Of Sunshine,Natasha Bedingfield,222000,0,1,Old ✌️,spotify:track:49ppqF73rOz5Nyt1paRxIF
3,You Make Me Wanna...,My Way,Usher,219000,0,27,Old ✌️,spotify:track:47TqCCnEliDp8NRDyIQoQq
4,Miss Independent,Year Of The Gentleman,Ne-Yo,232000,0,35,Old ✌️,spotify:track:34ceTg8ChN5HjrqiIYCn9Q
...,...,...,...,...,...,...,...,...
3458,Feeling Forever,Run Wild / Feeling Forever,PLS&TY,230000,0,4,❄🏠,spotify:track:0yAhB8wkhK7dNyLrzbkHpY
3459,I Found You,FRIENDS KEEP SECRETS,benny blanco,189000,0,5,❄🏠,spotify:track:5sdb5pMhcK44SSLsj1moUh
3460,Man X Woman,Majestic Casual - Chapter 2,Full Crate x Mar,279000,0,49,❄🏠,spotify:track:1adYhAnblyNTZtrPAuu5ZH
3461,You & Me,Settle (The Remixes),Disclosure,282000,0,8,❄🏠,spotify:track:1snNAXmmPXCn0dkF9DaPWw


In [34]:
full.Playlist.value_counts()

Now                     273
Canada                  254
Dreamville              239
Now 2                   194
Junkyard                191
Now 3                   188
✌️                      178
Junkyard 2              177
Now 4                   173
Country Side            146
Billboard               142
Now 5                   138
92 Til Infinity         133
✌️✌️                    126
❄🏠                      115
Throw it back (hood)    102
🇰🇷🇰🇵                     98
⚡ smooth                 97
JM and Friends           87
Hard                     64
⛄                        62
Frank                    53
Feelz                    43
Old ✌️                   43
Funky Old People         41
Millennium               30
🇹🇼                       25
Frat Rap                 21
Tropical                 13
International             6
Jazzy Fizzle              6
gpm                       5
Name: Playlist, dtype: int64

In [21]:
playlists = glob("./Playlists/*")
pre1 = {}
for p in playlists:
    pre1[p] = glob(p+"/*.csv")[0]
    
descriptions = {}
for k, v in pre1.items():
    descriptions[k.replace('./Playlists/', '')] = pd.read_csv(v)['Description'].iloc[0]

descriptions

{'Old ✌️': '00s rnb',
 'Millennium': '00s',
 '⛄': 'Heavenly',
 'Funky Old People': 'Oldies with some funkies',
 '✌️': nan,
 '⚡ smooth': nan,
 'Hard': nan,
 'Jazzy Fizzle': 'Piano',
 'Feelz': nan,
 'Junkyard 2': nan,
 'JM and Friends': 'Acoustic Life Vibes',
 'Throw it back (hood)': 'To that throwback 🇺🇸',
 'Frank': 'Ocean',
 'Frat Rap': 'Way back when',
 '🇰🇷🇰🇵': 'Korean',
 '✌️✌️': 2,
 'Tropical': nan,
 'Now': 'Music Now',
 'gpm': nan,
 '🇹🇼': 'Chinese',
 '92 Til Infinity': 'Mac and Friends',
 'Junkyard': 'Junky and everything else',
 'Now 3': nan,
 'Now 4': nan,
 'Now 5': nan,
 'Now 2': 'Part 2',
 'Country Side': 'Way out in the country',
 'Dreamville': 'Cole World and Friends',
 'Billboard': 'Pop',
 'International': 'World',
 'Canada': 'All the Canadians',
 '❄🏠': 'Chill house'}

In [3]:
### TESTING FOR FIRST ONE ###


# scope = 'playlist-modify-private'
# redirect_uri = "http://localhost:8080"
# token = util.prompt_for_user_token(config.user,
#                            scope,
#                            client_id=config.client_id,
#                            client_secret=config.client_secret,
#                            redirect_uri=redirect_uri)

# if token:
#     sp = spotipy.Spotify(auth=token)
#     sp.trace = False
#     new_playlist = sp.user_playlist_create(config.user, 'Old ✌️', public=False, collaborative=False, description='00s rnb')
#     results = sp.user_playlist_add_tracks(config.user, new_playlist['id'], playlist_songs)
# else:
#     print("Can't get token")

In [40]:
# test = full[full.Playlist == 'Old ✌️']
# tracks = test.spotify_id.to_list()
# sp.user_playlist_add_tracks(config.user, new_playlist['id'], tracks)

In [37]:
yesplaylists = ['⚡ smooth',
       'Hard', 'Jazzy Fizzle', 'Feelz', 'Junkyard 2', 'JM and Friends',
       'Throw it back (hood)', 'Frank', 'Frat Rap', '🇰🇷🇰🇵', '✌️✌️',
       'Tropical', 'Now', 'gpm', '🇹🇼', '92 Til Infinity', 'Junkyard',
       'Now 3', 'Now 4', 'Now 5', 'Now 2', 'Dreamville', 'Canada', '❄🏠']

info = {'Millennium': '00s',
 '⛄': 'Heavenly',
 'Funky Old People': 'Oldies with some funkies',
 '✌️': '',
 '⚡ smooth': '',
 'Hard': '',
 'Jazzy Fizzle': 'Piano',
 'Feelz': '',
 'Junkyard 2': '',
 'JM and Friends': 'Acoustic Life Vibes',
 'Throw it back (hood)': 'To that throwback 🇺🇸',
 'Frank': 'Ocean',
 'Frat Rap': 'Way back when',
 '🇰🇷🇰🇵': 'Korean',
 '✌️✌️': 2,
 'Tropical': '',
 'Now': 'Music Now',
 'gpm': '',
 '🇹🇼': 'Chinese',
 '92 Til Infinity': 'Mac and Friends',
 'Junkyard': 'Junky and everything else',
 'Now 3': '',
 'Now 4': '',
 'Now 5': '',
 'Now 2': 'Part 2',
 'Dreamville': 'Cole World and Friends',
 'Canada': 'All the Canadians',
 '❄🏠': 'Chill house'}

In [38]:
def populate_playlists(playlist):
    tracks = full[full.Playlist == playlist].spotify_id.to_list()
    playlist_name = playlist
    playlist_desc = info[playlist]
    
    new_playlist = sp.user_playlist_create(config.user, 
                                           playlist_name, 
                                           public=False, 
                                           collaborative=False, 
                                           description=playlist_desc)
    if len(tracks) <= 100:
        results = sp.user_playlist_add_tracks(config.user, new_playlist['id'], tracks)
    
    else:
        i = 0
        while i < len(tracks):
            results = sp.user_playlist_add_tracks(config.user, new_playlist['id'], tracks[i:i+100])
            i += 100
            

In [39]:
for p in yesplaylists:
    populate_playlists(p)

`user_playlist_create(user, name, public=True, collaborative=False, description='')`
Creates a playlist for a user

Parameters:
user - the id of the user
name - the name of the playlist
public - is the created playlist public
collaborative - is the created playlist collaborative
description - the description of the playlist



`playlist_add_items(playlist_id, items, position=None)`
Adds tracks/episodes to a playlist
Parameters:
playlist_id - the id of the playlist
items - a list of track/episode URIs, URLs or IDs
position - the position to add the tracks